In [ ]:
from pathlib import Path
from skimage import io
import numpy as np
import napari
from os.path import exists
from brainlit.utils.session import NeuroglancerSession
from cloudvolume.exceptions import SkeletonDecodeError
from napari_animation import AnimationWidget

In [ ]:
num = 9 #0,2,3,4,7,9
janelia = False

im_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/2018-08-01_" + str(num) + "_first10_quantitative.tif"
im = Path(im_path)
img = io.imread(im, plugin="tifffile")
print(f"Image shape: {img.shape}")

# read coords
if janelia:
    csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/voxel_coords.csv"
    coords = np.genfromtxt(csv_path, delimiter=',')
    coords = coords[10*num:10*(num+1)].astype(int)
    soma_coords = [list(coords[0,:])]
    axon_coords = [list(coords[-1,:])]
else:
    csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/my_points/points_" + str(num) + ".csv"
    coords = np.genfromtxt(csv_path, delimiter=',')
    coords = coords[1:].astype(int)
    coords = coords[:,1:]
    soma_coords = [list(coords[-1,:])]
    axon_coords = [list(coords[0,:])]

coords_list = list(coords)
coords_list = [list(c) for c in coords_list]
print(f"coords shape: {coords.shape}")

if num == 0:
    soma_coords += [[226, 643, 92], [233, 459, 120], [439, 500, 62]]
elif num == 7:
    soma_coords += [[335, 178, 39], [1, 418, 322]]

print(f"Soma coords: {soma_coords}")

In [134]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"

ngl_sess = NeuroglancerSession(mip = 0, url = dir, url_segments=dir_segments)

num_goal = 2
num = -1
skel_id = 0

while num < num_goal:
    try:
        ngl_sess.pull_vertex_list(skel_id, [0], 1)
        print(f"Skeleton # {skel_id} exists")
        num += 1
    except SkeletonDecodeError:
        print(f"#{skel_id} invalid")
    skel_id += 1

img, bbox, vox = ngl_sess.pull_vertex_list(skel_id-1, [0,1,2,3,4,5,6,7,8,9], [50,50,15])
print(f"Image shape: {img.shape}")
coords_list = vox
soma_coords = [coords_list[0]]
axon_coords = [coords_list[-1]]

#0 invalid
#1 invalid


Skeleton # 2 exists
#3 invalid
#4 invalid
#5 invalid
#6 invalid


Skeleton # 7 exists
#8 invalid
#9 invalid
#10 invalid


Skeleton # 11 exists






Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


































































































Image shape: (282, 308, 282)


In [135]:
from skimage import measure
import scipy.ndimage as ndi

new_true = np.multiply(np.array(coords_list), [0.3,0.3,1])
new_true_resample = resample(new_true)
new_true_resample = np.flip(new_true_resample, axis=0)

app_reflect_y = False

swcs = os.listdir("/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/gtree_2/")
swcs_valid = []
for swc in swcs:
    if swc.split("_")[0] == str(num):
        swcs_valid.append("/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/gtree_2/" + swc)

#
gtree_path, gtree_mask = read_swc(swcs_valid,
                    np.multiply(axon_coords[0], [0.3,0.3,1]), np.multiply(soma_coords[0], [0.3,0.3,1]), reflect_y=app_reflect_y)
#new_gtree = np.multiply(np.array(gtree_path), [0.3,0.3,1])
new_gtree = resample(gtree_path)

# gtree_mask_dilate = 0*gtree_mask
# for unq in tqdm(np.unique(gtree_mask)):
#     if unq == 0:
#         continue
#     mask_unq = gtree_mask == unq
#     mask_unq = ndi.binary_dilation(mask_unq, iterations=2)

#     gtree_mask_dilate[mask_unq > 0] += unq


viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img, scale=[0.3,0.3,1])
viewer.add_labels(gtree_mask)
#viewer.add_labels(gtree_mask_dilate>0, scale=[0.3,0.3,1])
viewer.add_shapes(new_gtree, shape_type='path', edge_width=1, edge_color='red', name="gtree")
viewer.add_shapes(new_true_resample, shape_type='path', edge_width=1, edge_color='green', name="truth")
viewer.add_points([axon_coords[0], soma_coords[0]], size=2, scale=[0.3,0.3,1])
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area="right")
viewer.camera.angles = [0, 90, 0]



100%|██████████| 42/42 [00:00<00:00, 68839.69it/s]


Downloading: 100%|██████████| 72/72 [00:02<00:00, 35.92it/s]
/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/napari_animation/_qt/keyframeslist_widget.py:156: FutureWarning: Themes were changed to use evented model with Pydantic's color type rather than the `rgb(x, y, z)`. You can get the old color by calling `color.as_rgb()`. The `as_dict=True` option will be removed in 0.X.X
  self.setStyleSheet(template(qss_template, **get_theme(theme_name)))


In [ ]:
manual_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/manual/" + str(num) + ".swc"
coords_list = []
with open(manual_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line[0] != "#":
            parts = line.split()
            coord = [float(parts[p]) for p in range(2,5)]
            coord = [int(c) for c in coord]
            coord.reverse()
            coords_list.append(coord)
coords_list.reverse()
coords_list = np.array(coords_list)
soma_coords = [coords_list[0]]
axon_coords = [coords_list[-1]]


In [38]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img, scale=[0.3,0.3,1])
viewer.add_shapes(coords_list, shape_type='path', edge_width=2, edge_color='green', name="truth", scale=[0.3,0.3,1])
#viewer.add_shapes(new_app, shape_type='path', edge_width=2, edge_color='red', name="truth")

viewer.add_points(soma_coords, face_color='orange', size=8, scale=[0.3,0.3,1])
viewer.add_points(axon_coords, face_color='red', size=8, scale=[0.3,0.3,1])
viewer.camera.angles = [0, -90, -90]
napari.run()


# Save as zarr

In [ ]:
import zarr

In [ ]:
zarr_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/zarr/" + str(num) + ".zarr"

z = zarr.zeros(img.shape, chunks=img.shape, dtype=img.dtype)
z = img
zarr.save(zarr_path, z)

# Make fragments

In [ ]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [ ]:
sg = state_generation(image_path="/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/zarr/" + str(num) + ".zarr",
        ilastik_program_path="/Applications/ilastik-1.3.3post3-OSX.app/Contents/ilastik-release/run_ilastik.sh",
        ilastik_project_path="/Users/thomasathey/Documents/mimlab/mouselight/octopus_experiment/octopus_exp.ilp",
        chunk_size=img.shape,
        soma_coords=soma_coords)

In [ ]:
sg.predict(data_bin="/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/zarr/misc/")

In [ ]:
sg.compute_frags()

In [ ]:
sg.compute_soma_lbls()

In [ ]:
sg.compute_image_tiered()

In [ ]:
sg.compute_states()

In [ ]:
sg.compute_edge_weights()
viterbrain = sg.viterbrain

## Compute shortest path

In [ ]:
import pickle

with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/zarr/' + str(num) + '_viterbrain.pickle', 'rb') as handle:
    viterbrain = pickle.load(handle)

In [ ]:
vb_path = viterbrain.shortest_path(axon_coords[0], soma_coords[0])
vb_path = [list(coord) for coord in vb_path]

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img)
viewer.add_shapes(vb_path, shape_type='path', edge_width=2, edge_color='red', name="viterbrain")
viewer.add_shapes(coords_list, shape_type='path', edge_width=2, edge_color='green', name="truth")

## Load other paths and resample

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
from cloudvolume import Skeleton
import networkx as nx
from tqdm import tqdm
from brainlit.viz import Bresenham3D
import similaritymeasures

In [50]:
def read_swc(file_paths, start_pt, end_pt, switch_axes = True, reflect_y = True, factor = [1,1,1]):
    cur_base = 0
    vert_cumulative = []
    g = nx.Graph()
    mask = 0*img

    for i, file_path in enumerate(file_paths):
        with open(file_path, "r") as f:
            swc = f.read()
        skel = Skeleton.from_swc(swc)

        verts = np.array(skel.vertices)

        if switch_axes:
            verts[:, [0,2]] = verts[:, [2,0]]
        if reflect_y:
            verts[:,1] = im_og.shape[1] - 1 - verts[:,1]
        verts = np.multiply(verts, factor)
        skel.vertices = verts
        vert_cumulative.append(verts)

        num_verts = skel.vertices.shape[0]
        g.add_nodes_from(np.arange(num_verts)+cur_base)
        for e in tqdm(skel.edges):
            g.add_edge(e[0]+cur_base,e[1]+cur_base)

        cur_base += num_verts

        #make mask
        skel_mask = 0*img

        for edge in tqdm(skel.edges, desc='Drawing edges...'):
            pt1 = skel.vertices[edge[0],:].astype(int)
            pt2 = skel.vertices[edge[1],:].astype(int)
            xs, ys, zs = Bresenham3D(int(pt1[0]), int(pt1[1]), int(pt1[2]),int(pt2[0]), int(pt2[1]), int(pt2[2]))
            skel_mask[xs, ys, zs] = 1
        
        mask[skel_mask > 0] = i+1

    vert_cumulative = np.concatenate(vert_cumulative, axis=0)
    # find path
    amins, _ = pairwise_distances_argmin_min(np.array([end_pt]), vert_cumulative)
    soma_id = amins[0]

    amins, _ = pairwise_distances_argmin_min(np.array([start_pt]), vert_cumulative)
    axon_id = amins[0]
        
    try:
        graph_path = nx.shortest_path(g, source=axon_id, target=soma_id)
    except:
        graph_path = [axon_id]

    path = []
    for id in graph_path:
        path.append(vert_cumulative[id, :])
    path = np.array(path)
    path = np.concatenate(([start_pt], path, [end_pt]), axis=0)

    return path, mask

def resample(path, spacing = 1):
    new_path = []
    for n in np.arange(path.shape[0]):
        pt1 = path[n-1:n,:]
        pt2 = path[n:n+1,:]

        new_path.append(pt1)
        dist = np.linalg.norm(pt1-pt2)

        if dist > 1:
            ts = np.arange(0, dist , 5)
            mid = np.zeros((len(ts)-1,3))
            for i,t in enumerate(ts[1:]):
                mid[i,:] = pt1 + (t/dist)*(pt2 -  pt1)
            new_path.append(mid)
    new_path.append(pt2)
    new_path = np.concatenate(new_path)
    return new_path

def sd(pts1, pts2, substantial = False, verbose=False):
    _, dists1 = pairwise_distances_argmin_min(pts1, pts2)
    _, dists2 = pairwise_distances_argmin_min(pts2, pts1)
    if verbose:
        print(dists1)
        print(dists2)
    if substantial:
        ddiv1 = np.mean(dists1[dists1 > 2])
        ddiv2 = np.mean(dists2[dists2 > 2])
        return np.mean([ddiv1, ddiv2])
    else:
        ddiv1 = np.mean(dists1)
        ddiv2 = np.mean(dists2)
        return np.mean([ddiv1, ddiv2])

In [ ]:
new_true = np.multiply(np.array(coords_list), [0.3,0.3,1])
new_true_resample = resample(new_true)
new_true_resample = np.flip(new_true_resample, axis=0)
new_viterbi = np.multiply(np.array(vb_path), [0.3,0.3,1])
new_viterbi = resample(new_viterbi)

In [ ]:
app_reflect_y = False

ad_path, _ = read_swc(["/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/advantra/" + str(num) + ".tif_Advantra.swc"],
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_ad = np.multiply(np.array(ad_path), [0.3,0.3,1])
new_ad = resample(new_ad)

app_reflect_y = False

app_path, _ = read_swc(["/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/app2/" + str(num) + ".swc"],
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_app = np.multiply(np.array(app_path), [0.3,0.3,1])
new_app = resample(new_app)

app_reflect_y = False

snake_path, _ = read_swc(["/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/snake/" + str(num) + ".tif_snake.swc"],
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_snake = np.multiply(np.array(snake_path), [0.3,0.3,1])
new_snake = resample(new_snake)

app_reflect_y = False

swcs = os.listdir("/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/gtree/")
swcs_valid = []
for swc in swcs:
    if swc.split("_")[0] == str(num):
        swcs_valid.append("/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/gtree/" + swc)
gtree_path, gtree_mask = read_swc(swcs_valid,
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_gtree = np.multiply(np.array(gtree_path), [0.3,0.3,1])
new_gtree = resample(new_gtree)

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img, scale=[0.3,0.3,1])
viewer.add_shapes(new_viterbi, shape_type='path', edge_width=1, edge_color='red', name="viterbrain")
viewer.add_shapes(new_true_resample, shape_type='path', edge_width=1, edge_color='green', name="truth")
viewer.add_shapes(new_app, shape_type='path', edge_width=1, edge_color='blue', name="app2")
viewer.add_shapes(new_ad, shape_type='path', edge_width=1, edge_color='blue', name="advantra")
viewer.add_shapes(new_snake, shape_type='path', edge_width=1, edge_color='blue', name="snake")
#viewer.add_shapes(new_gtree, shape_type='path', edge_width=2, edge_color='orange', name="gtree")
animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area="right")

In [ ]:
print(f"Viterbi: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_viterbi)}, SSD: {sd(new_true_resample, new_viterbi, substantial = False)}")
print(f"APP2: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_app)}, SSD: {sd(new_true_resample, new_app, substantial = False)}")
print(f"Advantra: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_ad)}, SSD: {sd(new_true_resample, new_app, substantial = False)}")
print(f"gtree: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_gtree)}, SSD: {sd(new_true_resample, new_gtree, substantial = False)}")


## Read my points

In [ ]:
csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/manual/points_" + str(num) + ".csv"
coords = np.genfromtxt(csv_path, delimiter=',')
coords = coords[1:].astype(int)
coords = coords[:,1:]
soma_coords = [list(coords[-1,:])]
axon_coords = [list(coords[0,:])]

coords_list = list(coords)
coords_list = [list(c) for c in coords_list]

vb_path = viterbrain.shortest_path(axon_coords[0], soma_coords[0])
vb_path = [list(coord) for coord in vb_path]

new_true = np.multiply(np.array(coords_list), [0.3,0.3,1])
new_true_resample = resample(new_true)
new_viterbi = np.multiply(np.array(vb_path), [0.3,0.3,1])
new_viterbi = resample(new_viterbi)

app_reflect_y = False

ad_path, _ = read_swc("/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/advantra/" + str(num) + ".tif_Advantra.swc",
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_ad = np.multiply(np.array(ad_path), [0.3,0.3,1])
new_ad = resample(new_ad)

app_reflect_y = False

app_path, _ = read_swc("/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/app2/" + str(num) + ".swc",
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_app = np.multiply(np.array(app_path), [0.3,0.3,1])
new_app = resample(new_app)

In [ ]:
print(f"Viterbi: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_viterbi)}, SSD: {sd(new_true_resample, new_viterbi, substantial = False)}")
print(f"APP2: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_app)}, SSD: {sd(new_true_resample, new_app, substantial = False)}")
print(f"Advantra: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_ad)}, SSD: {sd(new_true_resample, new_app, substantial = False)}")
# print(f"gtree: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_gtree)}, SSD: {sd(new_true_resample, new_gtree, substantial = False)}")
